In [561]:
import numpy as np
import pandas as pd

#### Load the matrices

In [562]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

#### Generating random combination

In [563]:
num_sequences=20
start_node=0
end_node=16
battery_threshold=10

w1=0.5
w2=0.5

In [564]:
def generate_random_sequences(num_sequences, battery_threshold=25):
    sequences = []
    node_list = list(range(17))

    for _ in range(num_sequences):
        while True:
            shuffled_nodes = np.random.permutation(node_list[1:-1])
            sequence = [start_node] + shuffled_nodes.tolist() + [end_node]
            battery_level = 78
            valid_sequence = True
            
            for i in range(len(sequence) - 1):
                node1 = sequence[i]
                node2 = sequence[i + 1]
                
                if node1 == 1 or node1 == 2:
                    battery_level=78

                battery_consumption = battery_expenditure.iloc[node1, node2]
                battery_level -= battery_consumption
                if battery_level < battery_threshold:
                    valid_sequence = False
                    break
            
            if valid_sequence:
                sequences.append(sequence)
                break

    return sequences

In [565]:
random_seq=generate_random_sequences(num_sequences)
random_seq

[[0, 8, 3, 15, 1, 6, 7, 12, 2, 11, 10, 5, 13, 4, 9, 14, 16],
 [0, 13, 3, 12, 15, 8, 5, 4, 9, 14, 6, 11, 1, 10, 7, 2, 16],
 [0, 7, 13, 4, 1, 2, 5, 10, 12, 9, 14, 6, 11, 3, 8, 15, 16],
 [0, 11, 9, 10, 1, 7, 3, 12, 8, 13, 15, 4, 6, 5, 14, 2, 16],
 [0, 14, 13, 11, 4, 12, 7, 10, 1, 9, 5, 15, 8, 2, 3, 6, 16],
 [0, 2, 8, 6, 5, 3, 12, 15, 14, 9, 7, 10, 13, 4, 11, 1, 16],
 [0, 2, 1, 7, 5, 15, 14, 8, 9, 13, 4, 12, 11, 10, 3, 6, 16],
 [0, 5, 15, 1, 8, 13, 11, 10, 9, 3, 2, 7, 4, 12, 14, 6, 16],
 [0, 13, 10, 4, 12, 7, 6, 15, 9, 14, 11, 3, 8, 1, 5, 2, 16],
 [0, 2, 6, 1, 8, 11, 5, 10, 9, 3, 7, 4, 15, 14, 12, 13, 16],
 [0, 12, 8, 6, 7, 14, 9, 11, 13, 3, 5, 2, 4, 15, 1, 10, 16],
 [0, 5, 12, 10, 9, 3, 8, 1, 6, 4, 14, 15, 2, 13, 11, 7, 16],
 [0, 6, 3, 12, 13, 4, 14, 7, 5, 15, 10, 1, 11, 9, 8, 2, 16],
 [0, 5, 11, 7, 2, 4, 13, 1, 8, 15, 6, 12, 3, 14, 9, 10, 16],
 [0, 8, 5, 13, 15, 14, 9, 10, 6, 11, 7, 12, 1, 4, 2, 3, 16],
 [0, 15, 9, 10, 3, 12, 4, 8, 2, 11, 5, 13, 7, 6, 14, 1, 16],
 [0, 15, 9, 6, 12, 14, 1

In [576]:
def calculate_cost(sequence, time_matrix, distance_matrix, w1, w2):
    cost1 = 0
    cost2 = 0
    cost = 0

    for i in range(len(sequence) - 1):
        node1 = sequence[i]
        node2 = sequence[i + 1]

        cost1 += time_matrix.iloc[node1, node2]
        cost2 += distance_matrix.iloc[node1, node2]

        cost += (w1 * time_matrix.iloc[node1, node2]) + (w2 * distance_matrix.iloc[node1, node2])

    return pd.DataFrame({'Σt': [cost1], 'Σd': [cost2], 'w1*Σt+w2*Σd': [cost]})

In [577]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in random_seq]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [578]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,6.291472,303.2223,154.756886
1,4.446639,221.6379,113.042269
2,5.168361,251.1104,128.139381
3,4.716333,223.1205,113.918417
4,5.656306,277.2469,141.451603
5,3.886861,184.9079,94.397381
6,5.058833,244.1070,124.582917
7,5.502611,260.7588,133.130706
8,5.558667,272.3238,138.941233
9,4.947667,232.6120,118.779833


## Genetic Algorithm implementation

In [579]:
def initialize_population(population_size):
    population = generate_random_sequences(population_size)
    return population


In [580]:
def tournament_selection(population, fitness_values, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmin(tournament_fitness)]
        selected_parents.append(population[winner_index])
    return selected_parents

In [581]:
import random

def ordered_crossover(parent1, parent2):
    length = len(parent1)
    crossover_point = random.randint(1, length - 2)

    while True:
        child = parent1[:crossover_point] + parent2[crossover_point:]

        if not any(child[i] == child[i + 1] for i in range(len(child) - 1)):
            break 

        crossover_point = random.randint(1, length - 2)

    print("Crossing : ", parent1, parent2)
    print("Result     : ", child)

    return child


In [572]:
def shift_mutation(sequence):
    mutation_point1 = np.random.randint(1, len(sequence) - 2)
    mutation_point2 = np.random.randint(mutation_point1 + 1, len(sequence) - 1)
    mutated_sequence = sequence[:mutation_point1] + sequence[mutation_point2:] + sequence[mutation_point1:mutation_point2]
    mutated_sequence[0] = sequence[0]
    mutated_sequence[-1] = sequence[-1]

    print( " Original : ",sequence, " Mutated : ", mutated_sequence)
    return mutated_sequence

In [582]:
import random

def genetic_algorithm(num_generations, population_size, tournament_size, mutation_probability):
    population = initialize_population(population_size)
    
    for generation in range(num_generations):
        fitness_values = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0] for sequence in population]

        parents = tournament_selection(population, fitness_values, tournament_size)
        offspring = []

        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                child1 = ordered_crossover(parents[i], parents[i + 1])
                child2 = ordered_crossover(parents[i + 1], parents[i])
                offspring.extend([child1, child2])

        mutated_offspring = []
        for child in offspring:
            # Apply mutation with a certain probability
            if random.uniform(0, 1) < mutation_probability:
                mutated_child = shift_mutation(child)
                mutated_offspring.append(mutated_child)
            else:
                mutated_offspring.append(child)

        population = mutated_offspring

    best_sequence = min(population, key=lambda x: calculate_cost(x, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0])

    return best_sequence


In [584]:
best_sequence = genetic_algorithm(num_generations=100, population_size=50, tournament_size=5, mutation_probability=0.000001)
print("Best Sequence:", best_sequence)
print("Best Cost:", calculate_cost(best_sequence, time_matrix, distance_matrix, w1, w2))

Crossing :  [0, 6, 12, 9, 4, 10, 3, 14, 5, 7, 15, 2, 13, 8, 11, 1, 16] [0, 3, 1, 15, 7, 5, 14, 12, 4, 2, 8, 10, 9, 13, 6, 11, 16]
Result     :  [0, 6, 12, 9, 4, 10, 3, 12, 4, 2, 8, 10, 9, 13, 6, 11, 16]
Crossing :  [0, 3, 1, 15, 7, 5, 14, 12, 4, 2, 8, 10, 9, 13, 6, 11, 16] [0, 6, 12, 9, 4, 10, 3, 14, 5, 7, 15, 2, 13, 8, 11, 1, 16]
Result     :  [0, 3, 12, 9, 4, 10, 3, 14, 5, 7, 15, 2, 13, 8, 11, 1, 16]
Crossing :  [0, 2, 15, 12, 11, 13, 4, 5, 1, 6, 14, 8, 9, 10, 7, 3, 16] [0, 2, 14, 11, 5, 6, 7, 13, 1, 8, 15, 9, 12, 3, 10, 4, 16]
Result     :  [0, 2, 15, 12, 11, 13, 4, 5, 1, 6, 14, 8, 12, 3, 10, 4, 16]
Crossing :  [0, 2, 14, 11, 5, 6, 7, 13, 1, 8, 15, 9, 12, 3, 10, 4, 16] [0, 2, 15, 12, 11, 13, 4, 5, 1, 6, 14, 8, 9, 10, 7, 3, 16]
Result     :  [0, 2, 14, 11, 5, 6, 7, 13, 1, 6, 14, 8, 9, 10, 7, 3, 16]
Crossing :  [0, 12, 8, 6, 3, 5, 2, 11, 14, 10, 9, 7, 15, 1, 13, 4, 16] [0, 6, 12, 10, 15, 14, 13, 2, 9, 7, 1, 11, 3, 5, 8, 4, 16]
Result     :  [0, 12, 8, 10, 15, 14, 13, 2, 9, 7, 1, 11, 3